In [210]:
import pandas as pd
pd.options.mode.chained_assignment = None
import chardet
import numpy as np

In [211]:
anos_analisados = list(range(2008, 2023))

In [212]:
csv_vitibrasil_producao = pd.read_csv('../dados_originais/embrapa/Producao.csv', sep=";")
csv_vitibrasil_comercio = pd.read_csv('../dados_originais/embrapa/Comercio.csv', sep=";")
csv_vitibrasil_exportacao = pd.read_csv('../dados_originais/embrapa/ExpVinho.csv', sep=";")
csv_vitibrasil_importacao = pd.read_csv('../dados_originais/embrapa/ImpVinhos.csv', sep=";")
csv_vitibrasil_processamento = pd.read_csv('../dados_originais/embrapa/ProcessaViniferas.csv', sep=";")

In [213]:
def importacao_dados_inmet(csv_file_path : str):

  with open (csv_file_path, "rb") as file:
    bytes = file.read(100)

  encoding_file = chardet.detect(bytes)

  if encoding_file.get("encoding") == 'ascii':
    encoding_file = 'ISO-8859-1'
  else:
    encoding_file = encoding_file.get("encoding")

  return pd.read_csv(csv_file_path
                              , sep=";"
                              , encoding=encoding_file
                              , skiprows=8)

In [214]:
csv_inmet_por_ano = {}

for ano in anos_analisados:
  csv_inmet_por_ano[ano] = importacao_dados_inmet(f'../dados_originais/INMET/INMET_S_RS_A840_BENTO GONCALVES_01-01-{ano}_A_31-12-{ano}.CSV')

csv_inmet_por_ano[2019] = csv_inmet_por_ano[2019].rename(columns={'Hora UTC': 'HORA (UTC)'})
csv_inmet_por_ano[2019] = csv_inmet_por_ano[2019].rename(columns={'Data': 'DATA (YYYY-MM-DD)'})

csv_inmet_por_ano[2020] = csv_inmet_por_ano[2020].rename(columns={'Hora UTC': 'HORA (UTC)'})
csv_inmet_por_ano[2020] = csv_inmet_por_ano[2020].rename(columns={'Data': 'DATA (YYYY-MM-DD)'})
csv_inmet_por_ano[2020] = csv_inmet_por_ano[2020].rename(columns={'RADIACAO GLOBAL (Kj/m²)': 'RADIACAO GLOBAL (KJ/m²)'})

csv_inmet_por_ano[2021] = csv_inmet_por_ano[2021].rename(columns={'Hora UTC': 'HORA (UTC)'})
csv_inmet_por_ano[2021] = csv_inmet_por_ano[2021].rename(columns={'Data': 'DATA (YYYY-MM-DD)'})
csv_inmet_por_ano[2021] = csv_inmet_por_ano[2021].rename(columns={'RADIACAO GLOBAL (Kj/m²)': 'RADIACAO GLOBAL (KJ/m²)'})

csv_inmet_por_ano[2022] = csv_inmet_por_ano[2022].rename(columns={'Hora UTC': 'HORA (UTC)'})
csv_inmet_por_ano[2022] = csv_inmet_por_ano[2022].rename(columns={'Data': 'DATA (YYYY-MM-DD)'})
csv_inmet_por_ano[2022] = csv_inmet_por_ano[2022].rename(columns={'RADIACAO GLOBAL (Kj/m²)': 'RADIACAO GLOBAL (KJ/m²)'})

In [215]:

for ano in anos_analisados:
    if ano != anos_analisados[-1]:
        inicial = 2008
        final = ano+1
        if csv_inmet_por_ano[inicial].columns.tolist() == csv_inmet_por_ano[final].columns.tolist():
            print(inicial, "e", final, 'as colunas são iguais.')
        else:
            print(inicial, "e", final, 'as coluans são diferentes.')

2008 e 2009 as colunas são iguais.
2008 e 2010 as colunas são iguais.
2008 e 2011 as colunas são iguais.
2008 e 2012 as colunas são iguais.
2008 e 2013 as colunas são iguais.
2008 e 2014 as colunas são iguais.
2008 e 2015 as colunas são iguais.
2008 e 2016 as colunas são iguais.
2008 e 2017 as colunas são iguais.
2008 e 2018 as colunas são iguais.
2008 e 2019 as colunas são iguais.
2008 e 2020 as colunas são iguais.
2008 e 2021 as colunas são iguais.
2008 e 2022 as colunas são iguais.


In [216]:
for ano in anos_analisados:
    if ano >= 2019:
        csv_inmet_por_ano[ano]['DATA (YYYY-MM-DD)'] = pd.to_datetime(csv_inmet_por_ano[ano]['DATA (YYYY-MM-DD)'], format='%Y/%m/%d')

In [217]:
colunas_para_converter = ['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)',
    'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)',
    'PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)',
    'PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)',
    'RADIACAO GLOBAL (KJ/m²)',
    'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)',
    'TEMPERATURA DO PONTO DE ORVALHO (°C)',
    'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)',
    'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)',
    'TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)',
    'TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)',
    'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)',
    'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)',
    'UMIDADE RELATIVA DO AR, HORARIA (%)',
    'VENTO, DIREÇÃO HORARIA (gr) (° (gr))',
    'VENTO, RAJADA MAXIMA (m/s)',
    'VENTO, VELOCIDADE HORARIA (m/s)',
    'Unnamed: 19']

# Concatenar todos os DataFrames em um único DataFrame
inmet = pd.concat(csv_inmet_por_ano, ignore_index=True)
inmet['DATA (YYYY-MM-DD)'] = pd.to_datetime(inmet['DATA (YYYY-MM-DD)'])
for coluna in colunas_para_converter:
    inmet[coluna] = pd.to_numeric(inmet[coluna], errors='coerce')
inmet_filtrado = inmet[inmet['PRECIPITAÇÃO TOTAL, HORÁRIO (mm)'] != -9999.0]
inmet_filtrado.replace(-9999, np.nan, inplace=True)


In [218]:
agregacoes = {
    'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)': 'sum',
    'PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)': ['min', 'max', 'mean'],
    # 'PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)': 'mean',
    # 'PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)': 'mean',
    'RADIACAO GLOBAL (KJ/m²)': 'mean',
    'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)': ['min', 'max', 'mean'],
    'TEMPERATURA DO PONTO DE ORVALHO (°C)': ['min', 'max', 'mean'],
    # 'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)',
    # 'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)',
    # 'TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)',
    # 'TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)',
    # 'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)',
    # 'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)',
    'UMIDADE RELATIVA DO AR, HORARIA (%)': ['min', 'max', 'mean'],
    'VENTO, DIREÇÃO HORARIA (gr) (° (gr))': ['min', 'max', 'mean'],
    'VENTO, RAJADA MAXIMA (m/s)': ['min', 'max', 'mean'],
    'VENTO, VELOCIDADE HORARIA (m/s)': ['min', 'max', 'mean'],
    # 'Unnamed: 19': ['min', 'max', 'mean'],
}
inmet_por_dia = inmet_filtrado.groupby('DATA (YYYY-MM-DD)').agg(agregacoes)